In [ ]:
import sys
import os
from ruamel.yaml import YAML

from collections import OrderedDict

In [ ]:
def rewrite_file(filename):
    yaml = YAML()
    
    print('Rewriting:', filename)

    with open("../config/{}".format(filename), "r") as f:
        data = yaml.load(f.read())
        
    ### initial_robot_configuration

    def mod_pos(arr, offset, relative_z=False):
        res = arr[:]
        res[offset + 0] = arr[offset + 1] # Swap y & x
        res[offset + 1] = arr[offset + 0] * 1.42 # Swap x & y
        if not relative_z:
            res[offset + 2] += 0.32 # z-offset
        arr[:] = res # Store result on the array again

    s = data['initial_robot_configuration']
    s['com'][2] += 0.32

    for (k, v) in s['eef_pose'].items():
        mod_pos(v, 1)

    ### terrain_description
    for e in data['terrain_description']['regions'][0]:
        e[2] += 0.32

    ### contact_plan

    s = data['contact_plan']
    for (k, v) in s.items():
        if k.startswith('effcnt'):
            for e in v:
                mod_pos(e, 2)
        elif k.startswith('effvia'):
            for e in v:
                mod_pos(e, 1)
        else:
            raise 'Unknown contact entry'
        s[k] = v[:] # Create a new list to get bullet point like list styles.

    ### planner_variables

    s = data['planner_variables']
    mod_pos(s['com_displacement'], 0, relative_z=True)

    if 'default_joint_positions' in s:
        v = s['default_joint_positions']
        for i in range(len(v)):
            v[i] *= -1
    else:
        s['default_joint_positions'] = tuple([0.52, -1.04, 0.52, -1.04, 0.52, -1.04, 0.52, -1.04])


    for (k, v) in s['com_viapoints'].items():
        mod_pos(v, 1)

    for k in ['eff_offset_rf', 'eff_offset_lf', 'eff_offset_rh', 'eff_offset_lh']:
        mod_pos(s[k], 0, relative_z=True)

    for (k, v) in [(k, v) for (k, v) in s.items() if k.startswith('w_') and isinstance(v, list) and len(v) == 3]:
        mod_pos(v, 0, relative_z=True)

    ### solverlqr_dynamics

    if 'solverlqr_dynamics' in data:
        s = data['solverlqr_dynamics']
        for (k, v) in s['user_parameters'].items():
            mod_pos(v, 0, relative_z=True)

    ### DONE
#     os.rename("../config/{}".format(filename), "../config/old/{}".format(filename))

    out_filepath = "../config/new/{}".format(filename)
    with open(out_filepath, 'w') as outfile:
        yaml.dump(data, outfile)

In [ ]:
from os import listdir
from os.path import isfile, join
cfg_files = [f for f in listdir('../config/') if f.endswith('.yaml')]
cfg_files += ['other_demos/' + f for f in listdir('../config/other_demos') if f.endswith('.yaml')]
cfg_files += ['SAB_Demos/' + f for f in listdir('../config/SAB_Demos') if f.endswith('.yaml')]

In [ ]:
for f in cfg_files:
    rewrite_file(f)

In [ ]:
rewrite_file('../config/cfg_demo02_trot.yaml')
rewrite_file('../config/cfg_quadruped_standing.yaml')